Oct 18 - Work in progress
References:
https://int8.io/monte-carlo-tree-search-beginners-guide/
https://www.geeksforgeeks.org/ml-monte-carlo-tree-search-mcts/
https://www.reddit.com/r/singularity/comments/1cmcmy5/monte_monte_carlo_tree_search_with_llms_is_the/
Search: Games, Minimax, and Alpha-Beta: https://ocw.mit.edu/courses/6-034-artificial-intelligence-fall-2010/resources/lecture-6-search-games-minimax-and-alpha-beta/
https://courses.cs.washington.edu/courses/cse599i/18wi/resources/lecture19/lecture19.pdf
applications illustrated: https://www.informs-sim.org/wsc18papers/includes/files/021.pdf
tic-tak-toe: https://stackoverflow.com/questions/49456415/monte-carlo-tree-search-tic-tac-toe-poor-agent


MCTS
What is it for?
- To create a tree structure search algorithm that balances depth and breadth (exploitation and exploration)


How it works?
- Each node should be a value the agents get by traversing to that node, whatever value that can be.
- When traveling to each node, you simulate the expected returns of current path and compare it with rewards of best strategy so far.
- You should probably estimate current resources, too.

- The resources and rewards may or may not be the same thing. Likely you may gain resources as rewards, but very likely they are different things.
- The final result is to pick up a path with largest expected return, before the resources run out, or you use as little resources as possible.

Selection: the act of choosing a path based on current informaion (resources/rewards). My understanding is it start with a random strategy until you get to know a better one.

Expansion: go down to the end of current path

Simulation: est rewards based on expected return of current path

Backpropagation: ????


Where things may get tricky?
- It seems MCTS is computationally demanding
-

Why does it even work in many applications?
-
- Computation would not be a limitation for employijng such strategy in reality. In other words, simulation and computation enable calculaton of best strategy. <br><br><br>


The main concept of monte carlo tree search is a search. Search is a set of traversals down the game tree. Single traversal is a path from a root node (current game state) to a node that is not fully expanded. Node being not-fully expanded means at least one of its children is unvisited, not explored. Once not fully expanded node is encountered, one of its unvisited children is chosen to become a root node for a single playout/simulation. The result of the simulation is then propagated back up to the current tree root updating game tree nodes statistics. Once the search (constrained by time or computational power) terminates, the move is chosen based on the gathered statistics.

Lots of dots not clearly connected together, right?

Let’s try to ask crucial questions regarding the simplified description above to slowly understand all the pieces:

what are expanded or not fully unexpanded game tree nodes?
what it means ‘to traverse down‘ during search? How is the next (child) node selected?
what is a simulation?
what is the backpropagation?
what statistics are back-propagated and updated in expanded game tree nodes ?
How is the final move even chosen ?

In [ ]:
# Pseudocode for MCTS
class MCTS():

  # global variables
  def __init__(self,):
    self.resources  = 100
    self.rewards = [all_reward, reward_pathA, reward_pathB, ...] # should be a dict
    self.steps = [all_steps, step_pathA, step_pathB, ...] # should be a dict
    self.best_path = {} # best path in our knowledge. dict, node:reward

  def resource_monitor(self,):   # it's called every time the agent makes a move.
    if self.resources ==0:
      return "Game Over"
    self.resource -= 1   # there is no return so far, but when you call this global variable it always tells you current remain resources.

  def policy(self,)  # the method to decide next move. It starts with a random strategy, and then moves based on our calculation of best path
    reward_new_path = simulation(path, reward)
    reward_best_path = list(self.best_path.values).sum()
    if ward_new_path > reward_best_path:
      self.best_path = new_path # where you got this new path?
    return self.best_path

  def stats(self, ):
    expected_rewards
    expected_resouces

  def simultation(self,path, reward): # it's called when you need to cal expected rewards of a given path
    expected_reward = self.rewads[path].sum().self.steps[path] * self.resources # assume resources are measured in the same way as steps, how much more rewards you may get in a given path with the resources given.
    return expected_rewards


  def actions_in_node(self, node):
    self.resource_monitor()
    updating_best_path(path, reward)





